# Pipeline de dados - API RestCountries

In [1]:
import pandas as pd 
import requests
import sqlite3
import numpy as np
from plyer import notification
from datetime import datetime

## 00. Constantes e Funções auxiliares

In [2]:
DB_PATH = "restcountries.db"

In [3]:
def alerta(nivel, base, etapa):
    """
    Retorna um alerta no desktop

    :param nivel: Nível do alerta
    :param base: Base onde ocorreu a falha
    :param etapa: Etapa onde ocorreu a falha

    :type nivel: int
    :type base: str
    :type etapa: str
    """

    niveis = {1:"Baixo",
          2: "Médio",
          3: "Alto"}

    if nivel not in niveis:
        raise ValueError(f"Nível inválido. Deve ser 1, 2 ou 3")
    
    alerta = notification.notify(title=f"ATENÇÃO: Alerta {niveis[nivel]}",
                               message=f"Falha no carregamento da base {base} na etapa {etapa}\n{datetime.now()}")
    return alerta

def get_req(url, base="base"):
    """
    Retorna o JSON de uma requisição a uma API. Em caso de resposta diferente de 200, retorna um alerta.

    :param url: URL da API
    :param base: Nome da base que será criada

    :type url: str
    :type base: str
    """
    # Trata respostas diferentes da esperada (200)
    req = requests.get(url)
    if req.status_code != 200:
        return alerta(2, base, "Extração")
    
    else:
        return req.json()
    
def criar_db(caminho_db):
    """Cria um banco de dados SQLite no caminho especificado.
    
    :param caminho_db: string com o caminho completo para o arquivo do banco de dados.
    :return: a conexão com o banco de dados ou None em caso de falha.
    """
    conn = None
    try:
        conn = sqlite3.connect(caminho_db)
        print(f"Banco de dados criado e conectado com sucesso em {caminho_db}")
        return conn.close()
    except Error as e:
        print(f"Erro ao conectar ao banco de dados: {e}")
    

def tabelas_bd(path=DB_PATH):
    """Retorna todas as tabelas da DB especificada

    Args:
        path (str, optional): caminho da db. Padrão DB_PATH.

    Returns:
        pandas.DataFrame: Dataframe contendo as tabelas da DB
    """
    conn = sqlite3.connect(path)
    query = "SELECT name FROM sqlite_master WHERE type='table'"
    schema = pd.read_sql(query, conn)
    conn.close()
    
    return schema
    
def salva_bd(df, nome_tabela):
    """Salva o DataFrame como uma tabela na base de dados

    Args:
        df (pandas.DataFrame): Tabela a ser inserida na base de dados
        nome_tabela (str): Nome da tabela a ser inserida

    Returns:
        bool: True
    """
    conn = sqlite3.connect(DB_PATH)
    salva_df = df.to_sql(nome_tabela, conn, if_exists='replace', index=False)
    conn.close()

    return True

def carrega_bd(nome_tabela, path = DB_PATH):
    """Carrega a tabela referenciada na base de dados

    Args:
        nome_tabela (str): Nome da tabela
        path (str, optional): Caminho para a base de dados. Padrão DB_PATH.

    Returns:
        pandas.DataFrame: Tabela em formato de DataFrame
    """
    conn = sqlite3.connect(path)
    query = f"SELECT * FROM {nome_tabela}"
    df_query = pd.read_sql(query, conn)
    conn.close()

    return df_query

    

## 01. Premissas para extração
Para esta atividade, utilizaremos a API REST Countries

Vamos extrair 3 tabelas a partir da requisição

In [4]:
# Definimos a url e executamos a request
url = "https://restcountries.com/v3.1/all"
req = get_req(url)

In [5]:
# Análise dos dados da tabela

print(f"Entradas na tabela: {len(req)}")

dados_disponiveis = list(req[0].keys())
print(f"Dados disponíveis: {dados_disponiveis}")



Entradas na tabela: 250
Dados disponíveis: ['name', 'tld', 'cca2', 'ccn3', 'cca3', 'cioc', 'independent', 'status', 'unMember', 'currencies', 'idd', 'capital', 'altSpellings', 'region', 'subregion', 'languages', 'translations', 'latlng', 'landlocked', 'area', 'demonyms', 'flag', 'maps', 'population', 'gini', 'fifa', 'car', 'timezones', 'continents', 'flags', 'coatOfArms', 'startOfWeek', 'capitalInfo', 'postalCode']


## 02. Extração das tabelas 

### 02.00 - Tabela 0
TUDO!

A própria requisição inteiramente transformada em um DataFrame

In [6]:
def extract_all():
    df_req = pd.DataFrame(req)
    return df_req

### 02.01 - Tabela I

Moedas do mundo


Passo 0: quais as informações relativas a moeda que temos na tabela principal?

In [7]:
def extract_currencies():
    df_currencies = df_req[["currencies"]]
    return df_currencies

### 02.02 - Tabela II
Nomes oficiais, extensões e população

In [8]:
def extract_demographics():

    # Definindo as colunas
    country_id = [i["ccn3"] if "ccn3" in i else np.nan for i in req]
    names = [i["name"]["official"] for i in req]
    extensions = [i["area"] for i in req]
    population = [i["population"] for i in req]

    # Criando o dicionário que dará origem ao DataFrame

    dict_t2 = {"country_id": country_id,
            "name_official": names,
            "extension": extensions,
            "population": population}

    df = pd.DataFrame(dict_t2)
    df["population_density"] = df["population"]/df["extension"]
    return df


### 02.03 - Tabela III
Nomes

In [9]:
def extract_country_names():
    country_id = [i["ccn3"] if "ccn3" in i else np.nan for i in req]
    t1_names = [i["name"] for i in req]
    t1_regions = [i["region"] for i in req]
    # Tratamos o caso de não existir a chave "subregion" no JSON da requisição
    t1_subregions = [i["subregion"] if "subregion" in i else np.nan for i in req]

    df = pd.DataFrame({"country_id": country_id,
                    "name": t1_names,
                    "region": t1_regions,
                    "subregion": t1_subregions})
    
    
    return df


In [10]:
extract_country_names()

,country_id,name,region,subregion
0,196,"{'common': 'Cyprus', 'official': 'Republic of ...",Europe,Southern Europe
1,232,"{'common': 'Eritrea', 'official': 'State of Er...",Africa,Eastern Africa
2,430,"{'common': 'Liberia', 'official': 'Republic of...",Africa,Western Africa
3,060,"{'common': 'Bermuda', 'official': 'Bermuda', '...",Americas,North America
4,336,"{'common': 'Vatican City', 'official': 'Vatica...",Europe,Southern Europe
...,...,...,...,...
245,728,"{'common': 'South Sudan', 'official': 'Republi...",Africa,Middle Africa
246,340,"{'common': 'Honduras', 'official': 'Republic o...",Americas,Central America
247,670,"{'common': 'Saint Vincent and the Grenadines',...",Americas,Caribbean
248,144,"{'common': 'Sri Lanka', 'official': 'Democrati...",Asia,Southern Asia


## 03 - Tratamento das tabelas

### 03.01 - Tabela 01 - Moedas do Mundo

Queremos extrair todas as moedas existentes no mundo. Para isto, executaremos os seguintes passos:
- extrair as informações de cada dicionário presente em cada linha
- transformar cada uma das moedas em uma nova linha
- transformar cada uma das chaves de cada moeda em uma nova coluna (nome e símbolo)
- tratar missing values
- concatenar o resultado

In [11]:
def trata_currencies(df_req):
    """Tratamento da tabela de moedas

    Args:
        df_req (pandas.DataFrame): DataFrame de moedas da RestCountries

    Returns:
        pandas.DataFrame: DataFrame tratado
    """
    currencies_acro = df_req["currencies"].apply(pd.Series).melt(var_name="sigla_moeda", value_name="nome_moeda").dropna()
    currencies_name_symbol = currencies_acro[["nome_moeda"]]["nome_moeda"].apply(pd.Series)
    currencies = pd.concat([currencies_acro, currencies_name_symbol], axis=1).drop("nome_moeda", axis=1)
    currencies = currencies.drop_duplicates("name").reset_index(drop=True)
    return currencies

### 03.02 - Tabela 02 - Demografia

### 03.03 - Tabela 03 - Nomes de países

Nesta tabela, identificamos 2 possibilidades de tratamento:
- Precisamos "desempacotar" os dicionários (tratamento semelhante ao da tabela de moedas)
    - extrair as informações de cada dicionário presente em cada linha
    - transformar cada uma das moedas em uma nova linha
    - transformar cada uma das chaves de cada moeda em uma nova coluna (nome e símbolo)
    - tratar missing values
    - concatenar o resultado

  

In [12]:
def trata_paises(tabela_paises):
    """Tratamento da tabela de países

    Args:
        df_req (pandas.DataFrame): DataFrame de países da RestCountries

    Returns:
        pandas.DataFrame: DataFrame tratado
    """
    tabela_paises_nomes = tabela_paises["name"].apply(pd.Series)
    tabela_paises_nomes = tabela_paises_nomes[["common","official"]]
    tabela_paises_final = pd.concat([tabela_paises, tabela_paises_nomes], axis=1).drop("name", axis=1)

    return tabela_paises_final




### Apêndice
#### Ideias para o projeto:
- Fazer a tabela 1 extrair TODOS os dados da API, a princípio classificando-os pelo country_id
- Na tabela 3, extrair na verdade todas as moedas do mundo, com nomes, acrônimos e símbolos. Atribuir um id para cada moeda

## 04 - Execução

In [15]:
print("> Criando banco de dados")
criar_db(DB_PATH)
print ("> Iniciando etapa de extração")
df_req = extract_all()

######################################################
print("> Tabela - Moedas")
try:
        tabela_moedas = extract_currencies()
        tabela_moedas = trata_currencies(tabela_moedas)
        salva_bd(tabela_moedas,"currencies")
        print("> Tabela - Moedas criada")
except:
        alerta(nivel = 3, 
               base = "currencies", 
               etapa = "EXTRACAO")
        
######################################################
print("> Tabela - Demografia")
try:
        tabela_demografia = extract_demographics()
        salva_bd(tabela_demografia,"demographics")
        print("> Tabela - Demografia criada")
except:
        alerta(nivel = 3, 
               base = "demographics", 
               etapa = "EXTRACAO")
        
######################################################
print("> Tabela - Nomes")
try:
        tabela_paises = extract_country_names()
        tabela_paises = trata_paises(tabela_paises)
        salva_bd(tabela_paises,"country_names")
        print("> Tabela - Nomes de países")
except:
        alerta(nivel = 3, 
               base = "country_names", 
               etapa = "EXTRACAO")

print("PROCESSO FINALIZADO COM SUCESSO")


> Criando banco de dados
Banco de dados criado e conectado com sucesso em restcountries.db
> Iniciando etapa de extração
> Tabela - Moedas
> Tabela - Moedas criada
> Tabela - Demografia
> Tabela - Demografia criada
> Tabela - Nomes
> Tabela - Nomes de países
PROCESSO FINALIZADO COM SUCESSO
